In [ ]:
%pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com
Note: you may need to restart the kernel to use updated packages.


In [7]:
import gurobipy as gp
from gurobipy import GRB, quicksum
import numpy as np
import math

# DATA

You can see the below code also under src/common/data_gen/based

In [9]:
import math
from typing import List, Tuple

EARTH_RADIUS_KM = 6371
DENSITY = 1
INCREMENT_RATE = 0.03

COORDINATE_LIST = [
    (48.1374, 11.5754),  # depot
    (48.1755, 11.5518),
    (48.1340, 11.5676),
    (48.1114, 11.4703),
    (48.2648, 11.6713),
    (48.2489, 11.6532),
    (48.2474, 11.6310),
    (48.2123, 11.6279),
    (48.2038, 11.6133),
    (48.2012, 11.6146),
    (48.1832, 11.6077),
    (48.1792, 11.5999),
    (48.1753, 11.6031),
    (48.1672, 11.5909),
    (48.1632, 11.5869),
    (48.1565, 11.5840),
    (48.3321, 10.8957),
    (48.1436, 11.5779),
    (48.1257, 11.5506),
    (48.1170, 11.5358),
    (48.1152, 11.5198),
    (48.1160, 11.5022),
    (48.1231, 11.4840),
    (48.1811, 11.5115),
    (48.1833, 11.5316),
    (48.1861, 11.5468),
    (48.1713, 11.5729),
    (48.1667, 11.5782),
    (48.1296, 11.5584),
    (48.0768, 11.5120),
    (48.0884, 11.4810),
    (48.1330, 11.5317),
    (48.1363, 11.5532),
    (48.1403, 11.5600),
    (48.1392, 11.5662),
    (48.1356, 11.5989),
    (48.1533, 11.6203),
    (48.1354, 11.5019),
    (48.1360, 11.5382),
    (48.1274, 11.6050),
    (48.1207, 11.6200),
    (48.1012, 11.6462),
    (48.0890, 11.6451),
    (48.1334, 11.6906),
    (48.1287, 11.6835),
    (48.1124, 11.5878),
    (48.1129, 11.5928),
    (48.1155, 11.5797),
    (48.1266, 11.6338),
    (48.1198, 11.5768),
    (48.1457, 11.5653),
    (48.1621, 11.5687),
    (48.2106, 11.5722),
    (48.2115, 11.5132),
    (48.1701, 11.5244),
    (48.1479, 11.5570),
    (48.1130, 11.5716),
    (48.0972, 11.5793)
]


def guess_traffic_density() -> List[float]:
    """
    Gets traffic density for each hour in [9, 21)

    :return: Traffic density for each hour in [9, 21)
    """
    hourly_density_list = []
    for hour in range(9, 21):
        hour_index = hour - 7
        density_hour = DENSITY * (1 + hour_index * INCREMENT_RATE)
        hourly_density_list.append(density_hour)
    return hourly_density_list


def get_tdttm(initial_tts: List[float], hourly_traffic_densities: List[float]) -> List[List[float]]:
    """
    Calculates dynamic duration data from static one for a specific source

    :param initial_tts: Duration time data for a specific source for the initial hour
    :param hourly_traffic_densities: Traffic density for each hour in [9, 21)
    :return: Dynamic duration time data for a specific source
    """
    tdttm = []
    for initial_tt in initial_tts:
        current_tt_list = [initial_tt]  # generated based on real tt data
        for hourly_density in hourly_traffic_densities:
            current_tt_list.append(initial_tt*hourly_density)
        tdttm.append(current_tt_list)
    return tdttm


def degrees_to_radians(degrees: float) -> float:
    """
    Makes degrees to radians conversion

    :param degrees: A value in degrees
    :return: A value in radians
    """
    return degrees * math.pi / 180


def distance_in_km_between_coordinates(source: Tuple[float, float], destination: Tuple[float, float]) -> float:
    """
    Gets km distance between given two locations

    :param source: Coordinates of the source location
    :param destination: Coordinates of the destination location
    :return: Distance between given two locations
    """
    lat1, lon1 = source
    lat2, lon2 = destination
    d_lat = degrees_to_radians(lat2-lat1)
    d_lon = degrees_to_radians(lon2-lon1)
    lat1 = degrees_to_radians(lat1)
    lat2 = degrees_to_radians(lat2)
    a = math.sin(d_lat/2) * math.sin(d_lat/2) + \
        math.sin(d_lon/2) * math.sin(d_lon/2) * math.cos(lat1) * math.cos(lat2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return EARTH_RADIUS_KM * c


def get_time_data(per_km_time: float = 5) -> List[List[List[float]]]:
    """
    Gets dynamic duration time data

    :param per_km_time: Multiplier to calculate duration from distance in km
    :return: Dynamic duration time data
    """
    tt_list = []
    for src_coordinate in COORDINATE_LIST:
        current_tt_list = []
        for dest_coordinate in COORDINATE_LIST:
            cur_dist = distance_in_km_between_coordinates(src_coordinate, dest_coordinate)
            current_tt_list.append(cur_dist*per_km_time)
        tt_list.append(current_tt_list)
    hourly_traffic_densities = guess_traffic_density()
    time_data = []
    for tt_list_src in tt_list:
        updated_tt_list_src = get_tdttm(tt_list_src, hourly_traffic_densities)
        time_data.append(updated_tt_list_src)
    return time_data

# DYNAMIC

In [10]:
def vrp_dynamic_3(d, n, K, Q, TIME_LIMIT=30, UNITS = 60 , BIG_COST = 1000000):
  EPSILON = 1e-9 # for hour-time constraints
  H = len(d) # 12
  b = [0]
  for i in range(n):
    b.append(1)

  model = gp.Model("VRP Index 3 - Dynamic")
  model.setParam('TimeLimit', TIME_LIMIT*60)

  # Create variables
  x = model.addVars(n+1, n+1, K, vtype=GRB.BINARY, name="x")
  y = model.addVars(n+1, K, vtype=GRB.BINARY, name="y")
  z = model.addVars(n+1, name="z")
  # Departure time of depot for each cycle
  t_depot = model.addVars(K, name="t_depot")
  # Departure time of customers
  t_customers = model.addVars(n+1, name="t_customers")
  # Hour of the departure time of depot for each cycle
  h_depot = model.addVars(K, H, vtype=GRB.BINARY, name="h_depot")
  # Hour of the departure time of customers
  h_customers = model.addVars(n+1, H, vtype=GRB.BINARY, name="h_customers")
  # Multiplication of x and y
  xy = model.addVars(n+1, n+1, K, vtype=GRB.BINARY, name="xy")
  # Multiplication of x, y and h_depot
  xyh_depot = model.addVars(n+1, K, H, vtype=GRB.BINARY, name="xyh_depot")
  # Multiplication of x, y and h_customers
  xyh_customers = model.addVars(n+1, n+1, K, H, vtype=GRB.BINARY, name="xyh_customers")
  # Answer
  vrp_duration = model.addVar(name="vrp_duration")

  # Arrival constraints
  model.addConstrs((quicksum(x[city_1, city_2, k] for city_2 in range(n+1)) == y[city_1, k]
                      for city_1 in range(1, n+1)
                      for k in range(K)), name="arrival_customers")
  model.addConstrs((quicksum(x[0, city_2, k] for city_2 in range(n+1)) <= y[0, k]
                      for k in range(K)), name="arrival_depot")
  # Departure constraints
  model.addConstrs((quicksum(x[city_1, city_2, k] for city_1 in range(n+1)) == y[city_2, k]
                      for city_2 in range(1, n+1)
                      for k in range(K)), name="departure_customers")
  model.addConstrs((quicksum(x[city_1, 0, k] for city_1 in range(n+1)) <= y[0, k]
                      for k in range(K)), name="departure_depot")

  # Subtourelimination constraints
  for city_1 in range(1, n+1):
      for city_2 in range(1, n+1):
          if city_1 != city_2:
              model.addConstr(z[city_1] - z[city_2] +
                              n * quicksum(x[city_1, city_2, k] for k in range(K)) <= n - 1)
  for city in range(n+1):
      model.addConstrs((x[city, city, k] == 0) for k in range(K))

  # Cluster
  model.addConstrs(quicksum(y[city, k] for k in range(K)) == 1
                   for city in range(1, n+1))
  model.addConstr(quicksum(y[0, k] for k in range(K)) == K)

  # Capacities
  model.addConstrs(quicksum(b[city]*y[city, k] for city in range(1, n+1)) <= Q
                   for k in range(K))

  # x & y constraints (multiplication)
  model.addConstrs((x[city_1, city_2, k] * y[city_1, k] == xy[city_1, city_2, k])
                   for city_1 in range(n+1) for city_2 in range(n+1) for k in range(K))

  # x & y & h_depot constraints (multiplication)
  model.addConstrs((xy[0, city_2, k] * h_depot[k, hour] == xyh_depot[city_2, k, hour])
                   for city_2 in range(n+1) for k in range(K) for hour in range(H))

  # x & y & h_customers constraints (multiplication)
  model.addConstrs((xy[city_1, city_2, k] * h_customers[city_1, hour] == xyh_customers[city_1, city_2, k, hour])
                   for city_1 in range(1, n+1) for city_2 in range(n+1) for k in range(K) for hour in range(H))

  # Hour constraints (depot)
  # For each cycle, add boundary on the departure time of depot
  # 60*hour <= t < 60*(hour+1)
  # Cannot put strict < or >. Therefore, used EPSILON value 1e-9
  model.addConstrs((h_depot[k, hour]*UNITS*hour <= t_depot[k]) for k in range(K) for hour in range(H))
  model.addConstrs((h_depot[k, hour]*UNITS*(hour+1)-EPSILON + (1-h_depot[k, hour])*BIG_COST >= t_depot[k]) for k in range(K) for hour in range(H))
  # Obvious that there should be exactly one hour (time zone/slice)
  model.addConstrs(quicksum(h_depot[k, hour] for hour in range(H)) == 1 for k in range(K))
  # In the final situation, you can ignore h_depot and t_depot of empty cycles

  # Hour constraints (customers)
  # For each customer, add boundary on the departure time of customer
  # 60*hour <= t < 60*(hour+1)
  # Cannot put strict < or >. Therefore, used EPSILON value 1e-9
  model.addConstrs((h_customers[city, hour]*UNITS*hour <= t_customers[city]) for city in range(1, n+1) for hour in range(H))
  model.addConstrs((h_customers[city, hour]*UNITS*(hour+1)-EPSILON + (1-h_customers[city, hour])*BIG_COST >= t_customers[city]) for city in range(1, n+1) for hour in range(H))
  # Obvious that there should be exactly one hour (time zone/slice)
  model.addConstrs(quicksum(h_customers[city, hour] for hour in range(H)) == 1 for city in range(1, n+1))

  # Time constraints (depot)
  # Set the first departure time as 0
  model.addConstr(t_depot[0] == 0)
  # Departure time of depot for cycles, except the first one, is after departure time of the customer just comes before + time cost of edge
  model.addConstrs((quicksum(xyh_customers[city, 0, k_prev, hour] * (t_customers[city] + d[hour][city][0])
                    for city in range(1, n+1) for hour in range(H)) <= t_depot[k])
                    for k in range(1, K) for k_prev in range(k))
  # In the final situation, you can ignore h_depot and t_depot of empty cycles

  # Time constraints (customers)
  # Departure time of customer, is after departure time of the customer/depot just comes before + time cost of edge
  model.addConstrs((t_customers[city_2] ==
                    quicksum(xyh_depot[city_2, k, hour] * (t_depot[k] + d[hour][0][city_2]) for k in range(K) for hour in range(H))
                    +
                    quicksum(xyh_customers[city_1, city_2, k, hour] * (t_customers[city_1] + d[hour][city_1][city_2]) for k in range(K) for hour in range(H) for city_1 in range(1, n+1)))
                    for city_2 in range(1, n+1))

  # Final distance
  # Total duration can be fetched from existing customer->0 edges
  model.addConstrs((xyh_customers[city, 0, k, hour] * (t_customers[city] + d[hour][city][0]) <= vrp_duration)
                    for city in range(1, n+1) for k in range(K) for hour in range(H))

  # Set objective
  model.setObjective(vrp_duration, GRB.MINIMIZE)

  # Run optimization
  model.optimize()

  # Print solution

  print("x")
  solution_x = model.getAttr('x', x)
  for k in range(K):
    print(f"k = {k}")
    for city_1 in range(n+1):
        for city_2 in range(n+1):
            if solution_x[city_1, city_2, k] > 1e-4:
                print('%s -> %s' % (city_1, city_2))

  print("y")
  solution_y = model.getAttr('x', y)
  for k in range(K):
    print(f"k = {k}")
    for city in range(n+1):
        if solution_y[city, k] > 1e-4:
            print('%s' % (city))

  print("h_depot")
  solution_h_depot = model.getAttr('x', h_depot)
  for k in range(K):
      for hour in range(H):
          if solution_h_depot[k, hour] > 1e-4:
              print('%s , %s -> %g' % (k, hour, solution_h_depot[k, hour]))

  print("h_customers")
  solution_h_customers = model.getAttr('x', h_customers)
  for city in range(n+1):
      for hour in range(H):
          if solution_h_customers[city, hour] > 1e-4:
              print('%s , %s -> %g' % (city, hour, solution_h_customers[city, hour]))

  print("t_depot")
  solution_t_depot = model.getAttr('x', t_depot)
  for k in range(K):
    print('%s: %g' % (k, solution_t_depot[k]))

  print("t_customers")
  solution_t_customers = model.getAttr('x', t_customers)
  for city in range(1, n+1):
    print('%s: %g' % (city, solution_t_customers[city]))

  print("Objective: "+str(model.objVal))

In [1]:
n = 16
K = 5
Q = 5
per_km_time = 2
TIME_LIMIT = 25

In [12]:
BIG_COST = 1000000
d_dynamic_old = get_time_data(per_km_time)  # dynamic duration data
H = len(d_dynamic_old[0][0])  # hours (time zones/slices)
d_dynamic = []

# NxNx12 to 12xnxn conversion
for t in range(H):
    d_dynamic_new = []
    for i in range(n):
        d_dynamic_new_src = []
        for j in range(n):
            if i != j:
                d_dynamic_new_src.append(d_dynamic_old[i][j][t])
            else:
                d_dynamic_new_src.append(-BIG_COST)
        d_dynamic_new.append(d_dynamic_new_src)
    d_dynamic.append(d_dynamic_new)

n -= 1  # n is set to the number of customers, deduct 1

In [ ]:
vrp_dynamic_3(d=d_dynamic, n=n, K=K, Q=Q, TIME_LIMIT=TIME_LIMIT)

Set parameter TimeLimit to value 1800
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (win64)

CPU model: Intel(R) Core(TM) i7-10750H CPU @ 2.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 2075 rows, 45055 columns and 12691 nonzeros
Model fingerprint: 0x1e67c1ce
Model has 41565 quadratic constraints
Variable types: 66 continuous, 44989 integer (44989 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+06]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [8e-02, 1e+06]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+06]
Presolve added 279 rows and 0 columns
Presolve removed 0 rows and 3115 columns
Presolve time: 0.24s
Presolved: 261616 rows, 116010 columns, 678956 nonzeros
Variable types: 35554 continuous, 80456 integer (80456 binary)
